In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Import the necessary libraries

In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVC
from sklearn import model_selection

# 2. Preparation for loading the data

In [ ]:
TRAINING_FILE = "/kaggle/input/tabular-playground-series-may-2021/train.csv"
TEST_FILE = "/kaggle/input/tabular-playground-series-may-2021/test.csv"

# 3. Very brief EDA (exploratory data analysis) 

In [ ]:
df = pd.read_csv(TRAINING_FILE)

# Transform "target" variable into integer
df["target"] = df["target"].map({'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3})
# Removing "id" column
df = df.drop(columns=["id"])

# Plotting distribution of target variable
sns.countplot(x=df["target"]);

## As we can see, Class 2 is overrepresented, while Class 1 is underrepresented. Therefore we apply stratification.

# 4. Creating folds with StratifiedKFold

In [ ]:
df["kfold"] = -1
df = df.sample(frac=1).reset_index(drop=True)
# Using 20 splits to save time training the model
kf = model_selection.StratifiedKFold(n_splits=10)

for fold, (trn_, val_) in enumerate(kf.split(X=df, y=df.target.values)):
    df.loc[val_, "kfold"] = fold

# 5. Preparing and training the model

In [ ]:
svm_clf = SVC(kernel="rbf", probability=True)

In [ ]:
# df = train file
# df_test = test file

df_test_prep = pd.read_csv(TEST_FILE)
df_test = df_test_prep.drop(["id"], axis=1).values

# Use fold "2" as train file = 10%
df_train = df[df.kfold == 2].reset_index(drop=True)
# Use fold "3" as test file = 10% 
df_valid = df[df.kfold == 3].reset_index(drop=True)
       
x_train = df_train.drop(["target", "kfold"], axis=1).values
y_train = df_train.target.values
    
x_valid = df_valid.drop(["target", "kfold"], axis=1).values
y_valid = df_valid.target.values

In [ ]:
# TRAINING
svm_clf.fit(x_train, y_train)

In [ ]:
pred = svm_clf.predict_proba(df_test) 

y_test = pd.DataFrame(pred)
submission = y_test.copy()
submission.columns = ["Class_1", "Class_2", "Class_3", "Class_4"]
submission["id"] = df_test_prep["id"]

In [ ]:
submission.head(10)